#Problem

Each row in this (fictional) dataset contains information about a person who signed up for a demo of a software product. The machine learning objective is to build a classification model to predict whether or not a given user will "convert" and purchase the software. You should use the "train" dataset to train and cross-validate your models, and the "test" dataset to evaluate how well your final model performs.

Data Feature Descriptions:

<li>num_visits - the number of distinct sessions the user spent with the software</li>
<li>total_minutes_on_demo - the number of total minutes the user spent using the software</li>
<li>user_age - the age of the user in years</li>
<li>demo_source - how the user initially discovered the software</li>
<li>data_rows_input - how many rows of their own data the user uploaded to the software</li>
<li>users_in_network - how many people the user knows who already use the software</li>
<li>outreach_emails_sent - the number of emails sent to the user by the software company while the user is in the demo period</li>
<li>support_tickets_filed - the number of support requests sent by the user to the software company while using the demo</li>

Label:
<li>converted - boolean label of whether the user converted and purchased the software.</li>

## Import Python Libraries

Anaconda is a distribution of the Python and R programming languages for large-scale data processing, predictive analytics, and scientific computing. The open source version of Anaconda includes support for over 400 of the most popular Python packages for data science, including the packages used in this demo.

<li>https://www.continuum.io/why-anaconda</li>
<li>package list: https://docs.continuum.io/anaconda/pkg-docs</li>

## Load data

Load the csv data into a Pandas dataframe. Pandas is great for ETL and exploration of tabular data (tsv, csv, psv format) and dictionary (or json) data.

In [ ]:
import pandas as pd
train_df = pd.read_csv('meetup_train.csv')
test_df = pd.read_csv('meetup_test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

#### Get the feature types and count.  Check for null values.

In [ ]:
# Objects in this dataset refer to string data types. In Pandas, mixed type columns will also be labeled object.
train_df.info()

#### Get descriptive statistics for the numeric features

In [ ]:
# The "converted" column is the label column - so we now know the overall distribution of wins and losses.
train_df.describe()

#### Get the number of unique items in the categorical features

In [ ]:
categorical_features = ['location', 'demo_source', 'operating_system']
for c in categorical_features:
    n = train_df[c].nunique()
    print c, 'has', n, 'unique items'

## Explore and Visualize the Data

#### Use Seaborn statistical data visualization python library
https://web.stanford.edu/~mwaskom/software/seaborn/index.html

In [ ]:
import seaborn as sns
sns.set(style="ticks")
sns.set_context(context="talk")
%matplotlib inline

ax = sns.countplot(x='demo_source', data=train_df, hue='converted', palette='Set2')
sns.despine()

In [ ]:
ax = sns.countplot(x='operating_system', data=train_df, hue='converted', palette='Set2')
sns.despine()

In [ ]:
g = sns.factorplot("converted", col="location", col_wrap=6, data=train_df, 
               kind="count", size=2.9, aspect=.8, palette="Set2")

In [ ]:
g = sns.factorplot(x="total_minutes_on_demo", y="demo_source", 
                   hue="converted", row="operating_system",
                   data=train_df, orient="h", size=2.5, aspect=3.5, palette="Set2",
                   kind="violin", split=True, bw=.2)

In [ ]:
g = sns.factorplot(x="data_rows_input", y="demo_source", 
                   hue="converted", row="operating_system",
                   data=train_df, orient="h", size=2.5, aspect=3.5, palette="Set2",
                   kind="violin", split=True, bw=.2)

In [ ]:
sns.set_context(context="notebook")
g = sns.pairplot(train_df[['num_visits', 'total_minutes_on_demo', 'user_age',
                           'data_rows_input', 'users_in_network', 'outreach_emails_sent',
                           'support_tickets_filed']], size=2)

In [ ]:
sns.set_context(context="talk")
ax = sns.jointplot(x="users_in_network", y="user_age", data=train_df, kind="scatter", size=8)

## Featurize the Data

In [ ]:
# convert the "converted" label column to numpy arrays
train_converted = train_df.pop('converted').values
test_converted = test_df.pop('converted').values

# drop location (not informative) features from training and test dataframes
train_df.drop('location', axis=1, inplace=True)
test_df.drop('location', axis=1, inplace=True)

# transform the categorical features to binary features
train_dummies_df = pd.get_dummies(train_df)
test_dummies_df = pd.get_dummies(test_df)

# get the feature names - this will be useful for the model visualization and feature analysis
features = train_dummies_df.columns.values

# convert the training and test dataframes to numpy arrays
train_data = train_dummies_df.values
test_data = test_dummies_df.values

print 'training data shape', train_data.shape
print 'test data shape', test_data.shape
print 'converted label data shape', train_converted.shape
print features

In [ ]:
train_dummies_df.head()

In [ ]:
test_dummies_df.head()

## Model the Data

Use classification tools in scikit-learn to make predictions.

Popular classification methods:
<li>Decision Tree</li>
<li>Logistic Regression</li>
<li>Support Vector Machines</li>
<li>Ensemble methods: Random Forest, Gradient Tree Boosting, AdaBoost</li>

Inputs:
<li>X: array of training data with shape = (n_samples, n_features)</li>
<li>y: array of labels with shape = (n_samples)</li>
<li>optional model parameters</li>

Key classifier methods:
<li>fit: Fit the model according to the given training data.</li>
<li>predict: Predict class labels for samples in X.</li>
<li>score: Returns the mean accuracy on the given test data and labels.</li>

http://scikit-learn.org

### Decision Tree

Goal: predicts the value of a target variable by learning simple decision rules inferred from the data features.

Benefits:
<li>Easily interpretable</li>
<li>Fast</li>
<li>Can handle both numeric and categorical data</li>
<li>Data does not need to be normalized</li>

Disadvantages:
<li>Prone to overfitting</li>
<li>Unstable to small data variations</li>

Parameters:
<li>criterion: The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. Default = 'gini'.</li>
<li>max_features: The number of features to consider when looking for the best split. Default = None.</li>
<li>max_depth: The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples. Default = None.</li>
<li>min_samples_split: The minimum number of samples required to split an internal node. Default = 2.</li>
<li>min_samples_leaf: The minimum number of samples required to be at a leaf node. Default = 1.</li>

http://scikit-learn.org/stable/modules/tree.html

https://en.wikipedia.org/wiki/Decision_tree_learning

In [ ]:
from sklearn import tree

# Data
X = train_data
y = train_converted

# Build classifier model with default parameters
dtc = tree.DecisionTreeClassifier()

# Fit model to all of the training data
dtc.fit(X, y)

# Mean accuracy of the training data and labels
print 'Training data accuracy score', dtc.score(X, y)

# Make predictions
predictions = dtc.predict(test_data)
print 'Example predictions', predictions[:5]

# Mean accuracy of the test data and labels
print 'Test data accuracy score', dtc.score(test_data, test_converted)

### Visualize the decision tree model

In [ ]:
import graphviz
    
tree.export_graphviz(dtc, out_file='tree.dot', feature_names=features)
with open("tree.dot") as f:
    dot_graph = f.read()

src = graphviz.Source(dot_graph)
src.view()

### Cross-validation

Cross-validation is the method of holding out a portion of your training data for model evaulation. There are many types of cross-validation, such as: k-fold, stratified k-fold, shuffle split.  

The goals of cross-validation are:
<li>Ensure that your model does not overfit the training data</li>
<li>Select optimal model parameters</li>

In [ ]:
import numpy as np
from sklearn.cross_validation import cross_val_score, StratifiedKFold

# Function used to print cross-validation scores
def training_score(est, X, y, cv):
    acc = cross_val_score(est, X, y, cv = cv, scoring='accuracy')
    roc = cross_val_score(est, X, y, cv = cv, scoring='roc_auc')
    print '5-fold Train CV | Accuracy:', round(np.mean(acc), 3),'+/-', \
    round(np.std(acc), 3),'| ROC AUC:', round(np.mean(roc), 3), '+/-', round(np.std(roc), 3)

In [ ]:
# data
X = train_data
y = train_converted

# build model
dtc = tree.DecisionTreeClassifier(criterion='gini', max_features=None, max_depth=3, 
                                  min_samples_split = 10, min_samples_leaf=5)

# cross-validation 
cv = StratifiedKFold(y, n_folds=5, shuffle=True)
training_score(dtc, X, y, cv)

In [ ]:
# Fit the model on all of the training data
dtc.fit(X, y)

# View the tree
tree.export_graphviz(dtc, out_file='tree.dot', feature_names=features)
with open("tree.dot") as f:
    dot_graph = f.read()

src = graphviz.Source(dot_graph)
src.view()

### Logistic Regression

Goal: Estimate the probability of a binary response based on one or more features. The probabilities describing the possible outcomes of a single trial are modeled using a logistic function.

Benefits:
<li>Fast</li>
<li>Stable</li>
<li>Can handle sparse data</li>

Disadvantages:
<li>Necessary to normalize data</li>
<li>Data must be numeric</li>

Parameters:
<li>penalty: Used to specify the norm used in the penalization ('l1' or 'l2'). Default = 'l2'.</li>
<li>C: Inverse of regularization strength, smaller values specify stronger regularization. It must be a positive float. Default = 1.0.</li>
<li>tol: Tolerance for stopping criteria. Default = 0.0001.</li>

In [ ]:
# Standardize features by removing the mean and scaling to unit variance
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train_data)
train_scaled = scaler.transform(train_data)
test_scaled = scaler.transform(test_data)

from sklearn import linear_model
X = train_scaled
y = train_converted

# Build model using default parameter values
lrc = linear_model.LogisticRegression()

# cross-validation 
cv = StratifiedKFold(y, n_folds=5, shuffle=True)
training_score(lrc, X, y, cv)

### Calculate Confusion Matrix

https://en.wikipedia.org/wiki/Confusion_matrix

In [ ]:
# Compute confusion matrix
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Split the data into a training set and a test set
X_train, X_test, Y_train, Y_test = train_test_split(X, y)

# Run classifier
Y_pred = lrc.fit(X_train, Y_train).predict(X_test)
cm = confusion_matrix(Y_test, Y_pred)

# Compute confusion matrix
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')
plt.show()

### Ensemble Model - Random Forest

Ensemble Model Goal: combine predictions of several models in order to improve the accuracy (decrease bias) and robustness (decrease variance) over a single model.

Random Forest: combines de-correlated trees, where each tree is built from a bootstrap sample and node splits are calculated from random feature subsets.

Parameters (in addition to decision tree parameters)
<li>n_estimators: The number of trees in the forest.</li>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X = train_scaled
y = train_converted

# Build model
rfc = RandomForestClassifier(criterion='gini', max_features=None, max_depth=3, 
                             min_samples_split = 10, min_samples_leaf=5,
                             n_estimators = 100,
                             n_jobs=-1)

# Cross-validation 
cv = StratifiedKFold(y, n_folds=5, shuffle=True)
training_score(rfc, X, y, cv)

#### Grid Search Parameters

<li>Parameters that are not directly learnt within estimators can be set by searching a parameter space for the best cross-validated performance score. </li>
<li>The drawback to grid search is that it can be computationally expensive and slow if you search a wide parameter space. </li>
<li>Another parameter search option is randomized search - sampling from a distribution of possible parameter values. </li> 

In [ ]:
# Grid Search Random Forest parameters
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score

X = train_scaled
y = train_converted

# Perform a grid search on random forest parameters
random_forest_grid = {'max_depth': [3, 5, None],
                      'max_features': ['sqrt', 'log2', None],
                      'min_samples_split': [10],
                      'min_samples_leaf': [5],
                      'n_estimators': [100],
                      'random_state': [1]}

rf_gridsearch = GridSearchCV(RandomForestClassifier(),
                             random_forest_grid,
                             n_jobs=-1,
                             verbose=True,
                             scoring='accuracy')
rf_gridsearch.fit(X, y)

print "Best parameters:", rf_gridsearch.best_params_
print ""

best_rf_model = rf_gridsearch.best_estimator_

# Cross Validate the best model
cv = StratifiedKFold(y, n_folds=5, shuffle=True)
training_score(best_rf_model, X, y, cv)

### Feature Importance

In [ ]:
# Retrain the best model on all of the training data
best_rf_model.fit(X, y)

# Calculate features importance, with higher values being more favorable
f = best_rf_model.feature_importances_

# Plot the feature importance
f_index = np.argsort(f)
sorted_features = f[f_index.tolist()]
sorted_features_names = features[f_index]
num_features = np.arange(1, (len(features)+1),1)
    
plt.bar(num_features, sorted_features, width=0.6, align='center',alpha=0.6)
plt.xticks(num_features, sorted_features_names, rotation=90)
plt.xlabel('Features')
plt.ylabel('Relative Feature Importance')
plt.title('Feature Importance')
plt.savefig('feature_importance.png')

## Evaluate your model

#### Receiver Operating Characteristic (ROC) Curve

The ROC curve is generated by varying the cut-off threshold over the whole range of the classifier's output,
thus generating a curve from many working points.

ROC curves typically feature true positive rate on the Y axis, and false positive rate on the X axis.
This means that the top left corner of the plot is the “ideal” point - a false positive rate of zero,
and a true positive rate of one. This is not very realistic, but it does mean that a larger area
under the curve (AUC) is usually better.

The “steepness” of ROC curves is also important, since it is ideal to maximize the true positive rate 
while minimizing the false positive rate.

The goal in ROC space is to be in the upper-left-hand corner.

The best AUC value is 1 and random classifier would have a value of 0.5.

This example shows the ROC response of different datasets, created from K-fold cross-validation. 
Taking all of these curves, it is possible to calculate the mean area under curve, and see the variance of 
the curve when the training set is split into different subsets. This roughly shows how the classifier output
is affected by changes in the training data, and how different the splits generated by K-fold cross-validation
are from one another.

https://en.wikipedia.org/wiki/Receiver_operating_characteristic

In [ ]:
import scipy as sp
from sklearn.metrics import auc, roc_curve

# Plot the ROC Curve
mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
tpr = dict()
fpr = dict()
roc_auc = dict()
plt.figure(num=1, figsize=(8,8))

X = train_scaled
y = train_converted
cv = StratifiedKFold(y, n_folds=5, shuffle=True)
for i, (train, test) in enumerate(cv):
    probas_ = best_rf_model.fit(X[train], y[train]).predict_proba(X[test])[:, 1]
    # Compute ROC curve and area under the curve
    fpr[i], tpr[i], thresholds1 = roc_curve(y[test], probas_)
    mean_tpr += sp.interp(mean_fpr, fpr[i], tpr[i])
    mean_tpr[0] = 0.0
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], lw=1, label='RF ROC fold %d (area = %0.2f)' % ((i+1), roc_auc[i]))

plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Random')

mean_tpr /= len(cv)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, 'k--',
         label='RF Mean ROC (area = %0.2f)' % mean_auc, lw=2)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate (Fall-out)')
plt.ylabel('True Positive Rate (Recall)')
plt.title('Receiver operating characteristic curve')
plt.legend(loc="lower right")
plt.show()

#### Precision - Recall (PR) Curve

PR curves feature Precision on the Y axis, and Recall on the X axis.

<li>Precision = TP/ TP + FP</li>
<li>Recall = TP/ TP + FN</li>

A high area under the PR curve represents both high recall and high precision, where high precision relates
to a low false positive rate, and high recall relates to a low false negative rate. High scores for both
show that the classifier is returning accurate results (high precision), as well as returning a majority
of all positive results (high recall).

When dealing with highly skewed classes, the PR curve is more informative than the ROC curve. 

The goal in PR space is to be in the upper-right-hand corner.

The best AUC value is 1 and the worst value is 0.

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

# Plot Precision-Recall curve
precision = dict()
recall = dict()
pr_auc = dict()
mean_recall = 0.0
mean_precision = np.linspace(0, 1, 100)
plt.figure(num=1, figsize=(8,8))

X = train_scaled
y = train_converted
cv = StratifiedKFold(y, n_folds=5, shuffle=True)
for i, (train, test) in enumerate(cv):
    probas_ = best_rf_model.fit(X[train], y[train]).predict_proba(X[test])[:, 1]
    # Compute PR curve and area under the curve
    precision[i], recall[i], thresholds2 = precision_recall_curve(y[test], probas_)
    mean_recall += sp.interp(mean_precision, precision[i], recall[i])
    pr_auc[i] = average_precision_score(y[test], probas_)
    plt.plot(precision[i], recall[i], lw=1, label='RF PR fold %d (area = %0.2f)' % ((i+1), pr_auc[i]))

mean_recall /= len(cv)
mean_recall[-1] = 0.0
mean_auc = auc(mean_precision, mean_recall)
plt.plot(mean_precision, mean_recall, 'k--',
         label='RF Mean PR (area = %0.2f)' % mean_auc, lw=2)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('True Positive Rate (Recall)')
plt.ylabel('Positive Predicitive Value (Precision)')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.show()

## Save Best Model

In [ ]:
import cPickle as pickle

def save_model(mdl, path):
    with open(path, 'w') as f:
        pickle.dump(mdl, f)

# pickle model
pickle_path = 'best_model.pkl'
mdl = best_rf_model.fit(X, y)
save_model(mdl, pickle_path)

## Make Predictions with Best Model

In [ ]:
# Load model
with open(pickle_path, 'r') as f:
    model = pickle.load(f)
    
# Make predictions
predictions = model.predict(test_scaled)
print 'Test Accuracy', accuracy_score(test_converted, predictions)